In [ ]:
import torch
import numpy as np
from PIL import Image
from utils import send_query, send_large_query


In [ ]:
embedding_database = torch.load('ALIGNED_EMBEDDING_DATABASE.pth')
all_paths = embedding_database['paths']
all_embs = embedding_database['embeddings']

In [ ]:
SQL_QUERY = """
    SELECT shibboleth, card_ID, aligned_ID
    FROM aligned
    JOIN user ON user.user_ID=aligned.user_ID;
"""
query_result = send_query(SQL_QUERY)
query_names = [q['shibboleth'] for q in query_result]
query_cards = [q['card_ID'] for q in query_result]
query_idxs = [int(q['aligned_ID'])-1 for q in query_result]


In [ ]:
# Putting some negative samples in the database as well
num_neg = 20000
negative_idxs = np.random.randint(0, len(all_embs), num_neg).tolist()
# Remove false negatives
negative_idxs = list(set(negative_idxs) - set(query_idxs))
num_neg = len(negative_idxs)
negative_names = ['<UNK>' for _ in range(num_neg)]
negative_cards = ['<UNK>' for _ in range(num_neg)]


In [ ]:
deploy_names = query_names + negative_names
deploy_cards = query_cards + negative_cards
deploy_idxs = query_idxs + negative_idxs
deploy_embs = all_embs[deploy_idxs]

deploy_database = {
    'names': deploy_names,
    'cards': deploy_cards,
    'embeddings': deploy_embs
}

torch.save(deploy_database, 'DEPLOY_DATABASE.tar')